## Before we start

Read mapping is performed routinely and people have pipelines that you will use 99% of the times.
In BigPurple for example the standard pipeline that people use is Igor's
[SNS](https://igordot.github.io/sns/). DO NOT go out there and start aligning from scratch!

## Required software

- [fastqc](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/): qc analysis
- [trim-galore](https://github.com/FelixKrueger/TrimGalore): for trimming reads if necessary (requires cutadapt)
- [multiqc](https://multiqc.info/): (optional) combine qc analyses
- [bwa](https://github.com/lh3/bwa): map reads
- [samtools](http://www.htslib.org/): view alingments
- [gnu-plot](http://www.gnuplot.info/): (Optional) plot alignment stats
- [igv](https://software.broadinstitute.org/software/igv/home): to view the resulting alignments

### Local Installation

To run this notebook locally you have to install the required software yourself.
To do so, first make sure you have added [bioconda](https://bioconda.github.io/) 
to you conda channels like:

```
conda config --add channels bioconda
```

This gives you access to many biology specific packages. 
Then create and activate a new environment like:

```
conda create --name bioinfo jupyterlab fastqc trim-galore multiqc bwa samtools gnuplot biopython matplotlib
conda activate bioinfo
```

where I included `jupyterlab` to produce this document.

IGV has to be run seperately on your local computer. You can download it 
[here](https://software.broadinstitute.org/software/igv/). 
For this notebook, I am going to use `igv-notebook` for demonstration purposes. 
To install that run `pip install igv-notebook` *after* you have activated the environment or
by uncoommenting the corresponding cell below.

### On BigPurple

If you are on BigPurple, you can reproduce most of the analysis without creating a conda environment by loading the necessary modules.

```
module load fastqc python/cpu/3.6.5 trimgalore bwa samtools gnuplot
```

There is no `multiqc` module as far as I can tell.

In [ ]:
# !pip install igv_notebook  #if needed

In [ ]:
import urllib.request
import gzip  # to open fastq.gz
from pathlib import Path  # to control files/dim

import igv_notebook

from Bio import SeqIO

# Standard Data Analysis
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

## Download Data

Data from ENCODE [ENCSR000EUA](https://www.encodeproject.org/experiments/ENCSR000EUA/).
There is a random sample of 1,000,000 reads from these data stored 
[here](https://genome.med.nyu.edu/public/tsirigoslab/teaching/bioinformatics/ENCSR000EUA/). 
If for some reason you want to work with the full set of reads uncomment the appropriate line 
in the following cell

In [ ]:
# make directory to store fastq files
fastq_dir = Path('fastq')
fastq_dir.mkdir(exist_ok=True)

# Download fastq files
sample_names = ['ENCFF001NQP', 'ENCFF001NQQ']

for sam in sample_names:
    fq = fastq_dir / Path(sam + '.fastq.gz')
    if not fq.exists():
        print(f'Downloading {sam}')
        file_url = f'https://genome.med.nyu.edu/public/tsirigoslab/teaching/bioinformatics/ENCSR000EUA/{sam}.fastq.gz'
        # file_url = f'https://www.encodeproject.org/files/{sam}/@@download/{sam}.fastq.gz'
        urllib.request.urlretrieve(file_url, fq)

In [ ]:
# if you get CERTIFICATE_VERIFY_FAILED error run this cell and retry

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

For this tutorial we are going to use a sub-sampled versions of these files provided in the course module.

In [ ]:
for fq in fastq_dir.iterdir():
    print(fq.name)

## FASTQ Quality Control

In [ ]:
qc_dir = Path('qc') / 'raw'
qc_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
!fastqc --threads 2 --outdir qc/raw fastq/ENCFF001NQP.fastq.gz  fastq/ENCFF001NQQ.fastq.gz

The reports are stored under `gc` directory. Links below:

- [ENCFF001NQP](./qc/raw/ENCFF001NQP_fastqc.html)
- [ENCFF001NQQ](./qc/raw/ENCFF001NQQ_fastqc.html)

Description of the different modules [here](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/)

Parameters:

- `cores`: parallel processing
- `phred64`: it's an Illumina 1.5 (see doc or fastqc)
- `fastqc`: run again fastqc after
- `illumina`: use illumina adapter

## Trim Reads

We are going to trim and filter reads to improve the quality

In [ ]:
!trim_galore --help

In [ ]:
!trim_galore --cores 2 --phred64 --fastqc --max_n 10  --illumina -o qc/clean fastq/ENCFF001NQP.fastq.gz fastq/ENCFF001NQQ.fastq.gz

We combine the reports for the new "cleaned-up" fastqs using 

In [ ]:
!multiqc -o qc/clean qc/clean

The resulting report is [here](qc/clean/multiqc_report.html)

#### Re-encoding fastq

The data we are dealing with are old, and use the illumina 1.5 Phred64 encoding. 
We are going to update them to Phred33 which is assumed by most modern software.

In [ ]:
fq_file = [Path(fq + '_trimmed.fq.gz') for fq in sample_names]
fq_dir = Path('fastq-clean')
fq_dir.mkdir(exist_ok=True)
qc_dir = Path('qc/clean')

for fq in fq_file:
    with gzip.open(qc_dir/fq, 'rt') as fin, gzip.open(fq_dir/fq, 'wt') as fout:
        SeqIO.write(iter(SeqIO.parse(fin, 'fastq-illumina')), fout, 'fastq')
    fq = qc_dir/fq
    fq.unlink()

## Alignment

We are going to use the BWA aligner. The alignment process requires 2 steps:

1. Indexing: preprocesses the genome
2. Mapping: reads to genome

The basic `bwa` commands are shown below

In [ ]:
!bwa 

### Indexing

1. Decide which genome we are going to map our reads to.
2. Download the genome. Possible options 
   [Ensembl](https://www.ensembl.org/index.html),
   [UCSC](https://hgdownload.soe.ucsc.edu/downloads.html),
   [iGenome](https://sapac.support.illumina.com/sequencing/sequencing_software/igenome.html)
3. Index the genome with `bwa index` in our case

Indexing takes time for large genomes (~1h for `mm10`) but it only has to be done once.
And for popular genomes you can find pre-computed indexes at iGenome (details
[here](http://igenomes.illumina.com.s3-website-us-east-1.amazonaws.com/README.txt)).
If you are on BigPurple there are indexes under `/gpfs/data/igorlab/ref/<genome>/<aligner>`
(replace `<genome>` and `<aligner>` obviously)

The index consists of multiple files all with the same prefix as the genome and different extensions

If you want to follow this tutorial exactly, download the reference (`ref`) from [here](https://genome.med.nyu.edu/public/tsirigoslab/teaching/bioinformatics/)

In [ ]:
%%bash

mkdir -p ref/mm10/BWAIndex

# Download reference genome
wget -c https://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/latest/mm10.fa.gz
mv mm10.fa.gz ref/mm10/genome.fa.gz

# create index
bwa index -p ref/mm10/BWAIndex/genome ref/mm10/genome.fa.gz

# unnecessary...
ln -s ../genome.fa.gz ref/mm10/BWAIndex/

The `ref/mm10` directory should have the following structure

In [ ]:
!tree ref/mm10/

### Mapping

Once we have the index we can map the reads quickly. We will use the default option `mem` for mapping. The syntax is

```
bwa mem index-prefix fastq-1 [fastq-2] > output.sam
```

In [ ]:
%%bash

prefix=ref/mm10/BWAIndex/genome
for fq in fastq-clean/*; do
    sam=${fq/%fq.gz/sam}    # change fq.gz to sam
    sam=$(basename "$sam")  # write in working directory
    echo Processing "$fq"
    bwa mem "$prefix" "$fq" > "$sam"
    echo
done

### SAM Files

#### Header

- Contains meta-data that help interpret and reproduce the alignment results.
- Header lines start with `@` and are organized as `@TAG:TYPE:VALUE` pairs.
  [Tags list](https://samtools.github.io/hts-specs/SAMtags.pdf)

Tags here:

- `@HQ`: File-level metadata (1 line, the 1st)
    + `VN`: Format version
    + `SO`: Sorting order of alignments. Valid values: unknown (default), 
       unsorted, queryname and coordinate.
- `@SQ`: Reference sequence dictionary. 
   The order of `@SQ` lines defines the alignment sorting order.
    + `SN`: Reference sequence name (chromosomes here)
    + `LN`: Reference sequence length
- `@PG`: Program (used to generate thef file)
    + `ID`: Program record identifier
    + `PN`: Program name
    + `PV`: Program Version
    + `PP`: Previous @PG-ID. Used to reconstruct the pipeline
    + `CL`: Command line


#### Body

- Rows = alignments 
- Columns = Alignment features

The mandatory columns are:

1.  `QNAME` the name of the read (query). Taken from fastq header line.
2.  `FLAGS` a number that in binary form answer some yes/no questions about the alingment.
    You can interpret the number [here](https://broadinstitute.github.io/picard/explain-flags.html)
3.  `RNAME` the name of the reference sequence
4.  `POS` the 1-based start position of alignment.
5.  `MAPQ` MAPping (aka alignment) Quality score (Phred scale). Originally developed for
    the [MAQ aligner](https://genome.cshlp.org/content/suppl/2008/09/26/gr.078212.108.DC1/maq-supp.pdf)
    but each aligner can estimate it differently.
6.  `CIGAR` (Concise Idiosyncratic Gapped Alignment Report) a compressed (RLE) 
    representation of the alignment.
7.  `RNEXT` only relevant for paired-end reads; `RNAME` of mate read
8.  `PNEXT` only relevant for paired-end reads; `POS` of mate read
9.  `TLEN` only relevant for paired-end reads; fragment length inferred from alignment
10. `SEQ` sequence (or reverse complement if the read is aligned to the reverse strand), aka 2nd line of fastq-entry.
11. `QUAL` quality sequence aka 4th line of fastq-entry


The first 40 lines of the file are shown below

In [ ]:
!head -40 ENCFF001NQP_trimmed.sam

#### Sort, Compress, and Index

- **Sort**: arrange alignments by order of appearance (`chr1` to `chrY` and position `1` to `LN`)
- **Compress**: SAM to BAM to save space
- **Index**: generate a BAM index for fast random access (to view with IGV later)

In [ ]:
%%bash

for sam in *.sam; do
    echo Processing "$sam"
    bam=${sam%.sam}.bam
    samtools view -h -b -S "$sam" | samtools sort > "$bam"
    samtools index "$bam"
done

### Alignment Stats

[samtools-flagstats](http://www.htslib.org/doc/samtools-flagstat.html)

In [ ]:
!samtools flagstats ENCFF001NQP_trimmed.bam

[samtools-stats](http://www.htslib.org/doc/samtools-stats.html)

In [ ]:
!samtools stats ENCFF001NQP_trimmed.bam > qc/clean/ENCFF001NQP_trimmed.bam.stats

In [ ]:
!grep '^SN' qc/clean/ENCFF001NQP_trimmed.bam.stats  # grep summary

And a report can be generated with `plot-bamstats` if you have `gnuplot`

In [ ]:
!plot-bamstats -p qc/aln/ qc/clean/ENCFF001NQP_trimmed.bam.stats

Results can be viewed [here](qc/aln/index.html). For a detailed descriptions see 
[samtools stats](http://www.htslib.org/doc/samtools-stats.htmlhttp://www.htslib.org/doc/samtools-stats.html)

## Viewing Alignment

IGV Alignments: https://software.broadinstitute.org/software/igv/AlignmentData

In [ ]:
igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "mm10",
        "locus": "chr2:98,661,570-98,667,811"
    }
)


b.load_track(
    {
        "name": "ENCFF001NQQ",
        "url": "/ENCFF001NQQ_trimmed.bam",
        "indexURL": "/ENCFF001NQQ_trimmed.bam.bai",
        "format": "bam",
        "type": "alignment"
    })


b.load_track(
    {
        "name": "ENCFF001NQP",
        "url": "/ENCFF001NQP_trimmed.bam",
        "indexURL": "/ENCFF001NQP_trimmed.bam.bai",
        "format": "bam",
        "type": "alignment"
    })
b.zoom_in()

Color Legend:

- If a nucleotide differs from the reference sequence in greater than 20% of quality weighted
  reads, IGV colors the bar in proportion to the read count of each base 
  (<span style='color:green;font-weight:bold'>A</span> , 
   <span style='color:blue;font-weight:bold'>C</span> , 
   <span style='color:orange;font-weight:bold'>G</span> , 
   <span style='color:red;font-weight:bold'>T</span> ).
- ![](https://software.broadinstitute.org/software/igv/sites/cancerinformatics.org.igv/files/images/insert_lrgr.jpg): for an inferred insert size that is larger than expected (possible evidence of a deletion)
- ![](https://software.broadinstitute.org/software/igv/sites/cancerinformatics.org.igv/files/images/insert_smlr.jpg): for an inferred insert size that is smaller than expected (possible evidence of an insertion)
- ![](https://software.broadinstitute.org/software/igv/sites/cancerinformatics.org.igv/files/images/chromosomecolors.jpg): for paired end reads that are coded by the chromosome on which their mates can be found

We observe a lot of low quality alignemnts (white boxes). We can filter them out

### Filtering 

There are many low quality reads `MAPQ=0` as we saw. Below is the distribution of MAPQ

In [ ]:
!samtools view ENCFF001NQQ_trimmed.bam | cut -f5 | sort | uniq -c | sed 's/ *//' > tmp.txt

In [ ]:
mapq = np.loadtxt('tmp.txt')
mapq = mapq[np.argsort(mapq[:,1]),:]
plt.plot(mapq[:,1], mapq[:,0])
plt.show()

We can remove them with `samtools view -q`

In [ ]:
%%bash

# -q: quality filter
# -h: include header
# -b: export bam

for f in *.bam; do
    echo Processing "$f"
    bam=${f%.bam}_q30.bam
    samtools view -h -b -q 30 "$f" > "$bam"
    samtools index "$bam"
done

And now the IGV session should be clean

In [ ]:
igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "mm10",
        "locus": "chr2:98,661,570-98,667,811"
    }
)


b.load_track(
    {
        "name": "ENCFF001NQQ",
        "url": "/ENCFF001NQQ_trimmed_q30.bam",
        "indexURL": "/ENCFF001NQQ_trimmed_q30.bam.bai",
        "format": "bam",
        "type": "alignment"
    })


b.load_track(
    {
        "name": "ENCFF001NQP",
        "url": "/ENCFF001NQP_trimmed_q30.bam",
        "indexURL": "/ENCFF001NQP_trimmed_q30.bam.bai",
        "format": "bam",
        "type": "alignment"
    })
b.zoom_in()